# Grid Search

De cele mai multe ori, modelele mai complexe au mai mulți hyperparametrii care se pot ajusta. Conceptul de Grid Search presupune antrenarea și validarea unui model pentru fiecare combinație posibilă a hyperparametrilor. Scikit-Learn are o clasă GridSearchCV care este capabilă de a testa prin cross validation diferite valori pentru hyperparametrii, valori care sunt oferite de către utilizatori sun formă de dicționar. Acest lucru permite partea de cross-validation și Grid Search să fie realizate pentru fiecare model în parte din sklearn

În continuare o să ne uităm peste partea practică în Jupyter Notebook și Python. O să scriem un cod prin care pentru început o să citim datele, o să le împărțim în train-test sets după care se va realiza partea de scalare a datelor.

In [1]:
# importing the libraries
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# read the data into a DataFrame
df = pd.read_csv('../data/DATA/Advertising.csv')

# split the date into Features and labels
X = df.drop('sales', axis=1)
y = df['sales']

# split the date into train-test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# create a scaler, fit the scaler and transform the data
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

Pentru a putea verifica partea de GridSearch trebuie să avem un model care are cel puțin doi hyperparametrii pe care îi putem modifica. Pentru asta o să ne folosim de ElasticNet Regression la care știm să avem ca și hyperparametrii alpha și l1_ratio (care reprezintă rația dintre L1 Regression și L2 Regression)

In [2]:
from sklearn.linear_model import ElasticNet

La acest model dorim să aflăm care este combinația cea mai bună dintre parametrul de alpha și l1_ratio prin care putem să avem cel mai bun model. Ceea ce nu dorim să facem este să creem un prin model, să îl antrenăm, să facem predicții, iar după ce am validat modelul să modificăm unul dintre acei hyperparametrii. Rularea manuală a acestui tip de verificare durează prea mult și necesită prea multă implicare umană. Mai există varianta de a scrie un for loop prin care să iterăm mai multe valori de hyperparametrii. Din moment ce această opțiune este mult mai eficientă, Scikit-Learn ne pune la dipoziție partea de GridSearch prin care se realizează acest procedeu de verificare și validare de duiferite valori ale hyperparametriilor unui model.

Primul pas este de a crea un model de ElasticNet simplu, fără niciun hyperparametru

In [3]:
model = ElasticNet()

Pasul doi este de a crea un grid al hyperparametrilor pe care dorim să îi verificăm care vine sub forma unui dicționar (de regulă poartă denumirea de param_grid). Pentru acest model o să verificăm valori pentru alpha și l1_ration. Acești hyperparametrii o să fie keys în dicționar, iar ca și values pentru aceste keys trebuie să avem valorile care dorim să fie testate, valori trecute într-o listă.

In [4]:
param_grid = {'alpha': [0.1, 1, 5, 50, 100], 
                'l1_ratio': [0.1, 0.5, 0.7, 0.95, 0.99, 1]}

Ce anume reprezintă dicționarul de mai sus? Modelul nostru de ElasticNet în cadrul GridSearch o să ia prima dată pentru hyperparaemtrul alpha valoarea 0.1 și pentru l1_ratio, valoarea 0.1 (primele valori din dicționar). O să ruleze un cross-validation cu aceste valori după care o să păstreze valoarea 0.1 la alpha, dar o să ia următoarea valoare pentru l1_ration, și anume 0.5. O să tot parcurgă aceste valori până ce creează modele cu toate combinațiile posibile. În continuare o să importăm GridSearchCV din sklearn.model_selection

In [5]:
from sklearn.model_selection import GridSearchCV

Ceea ce trebuie făcut în continuare este să creem un grid de modele. Acestea o să fie modelele pentru fiecare combinație posibilă a acelor hyperparametrii. Metoda GridSearchCV doar creează acest grid, nu și rulează partea de cross-validation pentru fiecare model în parte. Ca și metodă de utilizare este destul de asemănător cu cross_val_score și cross_validate. Ia un prim paraemtru de estimator care reprezintă modelul pentru care dorim să rulăm partea de GridSearch și Cross-Validation. Următorul parametru este cel de param_grid la care o să îi oferim ca și valoare acel dicționar pe care tocmai l-am creat mai sus. Mai există și parametrii de cv și scoring care sunt precum în partea de cross_val_score. Ce mai are în plus această metodă este un parametru de verbose care ia ca și valoare un integer. Acest integer reprezintă câte informații să se afișeze în momentul în care se rulează partea de GridSearchCV. Default este setat la 0, iar asta înseamnă că nu se afișează nicio informație. Dacă se setează la 1, se afișează ceva informații, dar la un nivel general. Cu cât valoarea este mai mare, cu atât sunt afișate mai multe informații.

In [8]:
grid_model = GridSearchCV(model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=0)

Se poate vedea că în cadrul metodei GridSearchCV nu am specificat un set de date pe care să ruleze. Acest lucru înseamnă că metoda nu antrenează niciun model, doar creează o instanță a modelelor pentru feicare combinație posibilă. Pentru a antrena aceste modele trebuie să apelăm metoda .fit() pentru variabila grid_model și să îi oferim ca și valoare setul de date de antrenare și labels

In [9]:
grid_model.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=ElasticNet(),
             param_grid={'alpha': [0.1, 1, 5, 50, 100],
                         'l1_ratio': [0.1, 0.5, 0.7, 0.95, 0.99, 1]},
             scoring='neg_mean_squared_error')

Ceea ce se returnează este o informație cum că modelul respectiv utilizează GridSearchCV pentru un model de ElasticNet() cu parametrii respectivi și ca și metric ceea ce este la parametrul de scoring. Dacă modificăm valoarea de la verbose să fie 1 o să avem mai multe informații afișate și returnate

In [10]:
grid_model = GridSearchCV(model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1)

In [11]:
grid_model.fit(X_train, y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


GridSearchCV(cv=5, estimator=ElasticNet(),
             param_grid={'alpha': [0.1, 1, 5, 50, 100],
                         'l1_ratio': [0.1, 0.5, 0.7, 0.95, 0.99, 1]},
             scoring='neg_mean_squared_error', verbose=1)

După ce am modificat valoarea de la verbose la 1 acuma ni-i s-a afișat și numărul total de iterații pe care trebuie să îl facă algorimtul pentru a putea verifica fiecare combinație posibilă. Este de menținut faptul că la un set de date cu extrem de multe valori, acest proces de GridSearchCV poate dura un timp îndelungat. După cum s-a văzut, în momentul în care am utilizat metoda GridSearchCV i-am atribuit un estimator. Metoda respectivă a creat mai mulți estimatori (mai multe modele) cu diferite valori pentru hyperparametrii. Scopul este de a afla cel mai bun estimator. Pentru asta putem utiliza atributul 'best_estimator_' ce aparține de variabile 'grid_model' (asta după ce am antrenat grid-ul respectiv)

In [12]:
grid_model.best_estimator_

ElasticNet(alpha=0.1, l1_ratio=1)

Ceea ce este rezultate reprezintă modelul cu parametrii care a scos cea mai bună performanță (cea mai mică valoare de la mean_squared_error). Modelul de ElasticNet() cel mai performant are valoarea de 0.1 la hyperparametrul alpha și 1 la l1_ratio. Mai putem să aflăm acei parametrii și dacă accesăm atributul de best_params_

In [13]:
grid_model.best_params_

{'alpha': 0.1, 'l1_ratio': 1}

Poate există anumite momente sau cazuri când se dorește să aflăm valoarea de la mean_squared_error care a fost returnată pentru fiecare model în parte. Pentru astfel de situații o să accesăm atributul de .cv_results_. Acesta o să returneze un dicționar de valori pe care putem să îl transformăm într-un DataFrame pentru o mai bună reprezentare a datelor

In [14]:
grid_model.cv_results_

{'mean_fit_time': array([0.00144215, 0.00144958, 0.00126038, 0.00091515, 0.00082397,
        0.00073414, 0.00062919, 0.00080452, 0.00096397, 0.00141211,
        0.00087042, 0.00080819, 0.00069709, 0.00073519, 0.00075545,
        0.00095959, 0.0006155 , 0.00063658, 0.00071802, 0.00065794,
        0.00064621, 0.00057878, 0.00058413, 0.00073938, 0.00068736,
        0.00090227, 0.0007771 , 0.00099621, 0.00094829, 0.00145807]),
 'std_fit_time': array([3.23336783e-04, 4.68838991e-04, 5.12022438e-04, 1.81374940e-04,
        1.26192321e-04, 7.09006347e-05, 1.65318831e-05, 2.14303457e-04,
        2.41007878e-04, 7.50462378e-04, 1.29251934e-04, 1.49145440e-04,
        2.11488972e-05, 1.29621286e-04, 2.03760424e-04, 1.43616624e-04,
        6.63704569e-05, 8.56162412e-05, 1.19744872e-04, 1.07602782e-04,
        6.67097741e-05, 2.07095598e-05, 2.21156848e-05, 1.42036928e-04,
        7.17460097e-05, 1.76191465e-04, 1.10432912e-04, 2.86063215e-04,
        1.54639298e-04, 1.87483954e-04]),
 'mean_scor

In [16]:
pd.DataFrame(grid_model.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001442,0.000323,0.000581,0.000103,0.1,0.1,"{'alpha': 0.1, 'l1_ratio': 0.1}",-3.453021,-1.405190,-5.789125,-2.187302,-4.645576,-3.496043,1.591601,6
1,0.001450,0.000469,0.000488,0.000096,0.1,0.5,"{'alpha': 0.1, 'l1_ratio': 0.5}",-3.325440,-1.427522,-5.595610,-2.163089,-4.451679,-3.392668,1.506827,5
2,0.001260,0.000512,0.000519,0.000176,0.1,0.7,"{'alpha': 0.1, 'l1_ratio': 0.7}",-3.269880,-1.442432,-5.502437,-2.163950,-4.356738,-3.347088,1.462765,4
3,0.000915,0.000181,0.000415,0.000083,0.1,0.95,"{'alpha': 0.1, 'l1_ratio': 0.95}",-3.213052,-1.472417,-5.396258,-2.177452,-4.241080,-3.300052,1.406248,3
4,0.000824,0.000126,0.000328,0.000007,0.1,0.99,"{'alpha': 0.1, 'l1_ratio': 0.99}",-3.208124,-1.478489,-5.380242,-2.181097,-4.222968,-3.294184,1.396953,2
5,0.000734,0.000071,0.000331,0.000020,0.1,1,"{'alpha': 0.1, 'l1_ratio': 1}",-3.206943,-1.480065,-5.376257,-2.182076,-4.218460,-3.292760,1.394613,1
6,0.000629,0.000017,0.000316,0.000031,1,0.1,"{'alpha': 1, 'l1_ratio': 0.1}",-9.827475,-5.261525,-11.875347,-7.449195,-8.542329,-8.591174,2.222939,12
7,0.000805,0.000214,0.000327,0.000038,1,0.5,"{'alpha': 1, 'l1_ratio': 0.5}",-8.707071,-4.214228,-10.879261,-6.204545,-7.173031,-7.435627,2.255532,11
8,0.000964,0.000241,0.000356,0.000045,1,0.7,"{'alpha': 1, 'l1_ratio': 0.7}",-7.920870,-3.549562,-10.024877,-5.379553,-6.324836,-6.639940,2.206213,10
9,0.001412,0.000750,0.000473,0.000124,1,0.95,"{'alpha': 1, 'l1_ratio': 0.95}",-6.729435,-2.591285,-8.709842,-4.156317,-5.329916,-5.503359,2.102835,9


În DataFrame-ul de mai sus avem toate informațiile despre fiecare model în parte. În cazul în care nu suntem mulțumiți de performanța modelului, atunci trebuie să modificăm valorile din variabila 'param_grid' și să selectăm alte valori pentru acei hyperparametrii. Dacă suntem mulțumuți de rezultat, atunci pe acest grid putem să apelăm direct metoda .predict(), iar Scikit-Learn știe să ia cel mai bun model și să facă predicții. Modelul respectiv este deja antrenat, nu mai trebuie să facem asta

In [17]:
y_pred = grid_model.predict(X_test)

In [18]:
from sklearn.metrics import mean_squared_error

RMSE = np.sqrt(mean_squared_error(y_pred, y_test))
RMSE

1.5451027933724908

In [20]:
param_grid = {'alpha': [0.01, 0.05, 0.1, 0.15, 0.2], 
                'l1_ratio': [0.90, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1]}

grid_model = GridSearchCV(model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1)
grid_model.fit(X_train, y_train)
grid_model.best_estimator_

Fitting 5 folds for each of 55 candidates, totalling 275 fits


ElasticNet(alpha=0.1, l1_ratio=1)

## Recapitulare

În partea aceasta am învățat despre GridSearchCV următoarele lucruri:

    1. De unde să importăm metoda GridSearchCv

        from sklearn.model_selection import GridSearchCV

    2. Care sunt parametrii de care are nevoie această metodă

        estimator = modelul pe care să îl folosim

        param_grid = un dicționar care reprezintă un grid de parametrii pentru testare

        cv = numărul de k-folds

        scoring = metrica folosită pentru partea de validare

        verbose = cantitaea de informații care să fie afișată când se rulează acest GridSearch

    3. Cum să creem un dicționar ce reprezintă grid-urile de testare ale modelului

        param_grid = {'alpha': [0.1, 0.5, 1, 10, 50, 100], 'l1_ratio': [0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.97, 0.99, 1]}

    4. Cum să utilizăm metoda GridSearchCV

        grid_model = GridSearchCV(model, param_grid=param_grid, cv=10, scoring='neg_mean_squared_error', verbose=1)
    
    5. Grid-ul returnat nu este antrenat, prin urmare trebuie să îl și antrenăm

        grid_model.fit(X_train, y_train)

    6. Cum să aflăm modelul cu cei mai buni parametrii (cel mai bun estimator)

        grid_model.best_estimator_

        grid_model.best_params_

    7. Cum să facem predicții cu cel mai bun estimator

        grid_model.predict(X_test) # ia automat cel mai bun estimator